In [1]:
import json
import gzip
import os
from datetime import timedelta, date
import pandas as pd

In [2]:
from paper.config import *

by_dir

'/Users/oyvsyo/proj/jooble/IR/jooble-ir-2018-paper/Data/by'

In [3]:
# by_dir = "/mnt/storage2/Data/by"
list_json = []

start_date = date(2017, 1, 1)
# end_date = date(2017, 1, 30)
end_date = date(2017, 1, 3)

date_range = pd.date_range(start_date, end_date)

#looping through all days from 2017_01_1 to 2017_01_30 with step = 1 day
for single_date in date_range:
    date_ = single_date.strftime("%Y-%m-%d")
        
    #!!!!CHANGE PATH!!!!!!
    file_job = os.path.join(by_dir, 'by_job_{}.json.gz'.format(date_))
    file_job_text = os.path.join(by_dir, 'by_job_text_{}.json.gz'.format(date_))

    #reading two files simuntaneously
    document_job, document_job_text = [],[]
    
    with gzip.open(file_job, "rb") as f1, gzip.open(file_job_text, "rb") as f2: 
        for line1, line2 in zip(f1, f2):
            vacation_job = json.loads(line1)
            vacation_job_text = json.loads(line2)
            document_job.append(vacation_job)
            document_job_text.append(vacation_job_text)

        #creating a list of dictionaries
        for i in range(len(document_job)):
            for j in range(len(document_job_text)):
                if document_job[i]['id'] == document_job_text[j]['id_job']:
                    data = {}
                    data['id'] = document_job[i]['id']
                    data['lang_text'] = document_job[i]['lang_text']
                    data['lang_title'] = document_job[i]['lang_title']
                    data['company_name'] = document_job[i]['company_name']
                    data['title'] = document_job[i]['title']
                    data['title_normalized'] = document_job[i]['title_normalized']
                    data['salary_val1'] = document_job[i]['salary_val1']
                    data['salary_val2'] = document_job[i]['salary_val2']
                    data['company_name'] = document_job[i]['company_name']
                    data['url'] = document_job[i]['url']
                    
                    data['text'] = document_job_text[j]['text']
                    list_json.append(data)
                    
print(len(list_json))

2333


In [ ]:
! cat /mnt/storage2/Git/jooble-ir-2018-paper/Data/by/by_job_2017-03-02.json.gz
! Data/by/


In [4]:
from paper.analyze import norm_stem_lang
from paper.analyze import normalize
from paper.utils import timed

@timed
def stemm_d(list_json):
    # stemmed = [norm_stem_lang(doc['title']+ ' ' + doc['text']).split() for doc in list_json]
    stemmed = []
    for doc in list_json:
        try:
            stemmed.append(norm_stem_lang(doc['title']+ ' ' + doc['text']))
        except:
            pass
    return stemmed
        
stemmed = stemm_d(list_json)

stemm_d: 49.57786202430725


In [5]:
len(stemmed)

2333

In [ ]:
# list_json[24]

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from redis import Redis
import scipy.sparse


tf_idf_vectorizer = TfidfVectorizer()
redis_conn = Redis(decode_responses=True)


def update_index_tfidf(docs, stemmed):
    """
    docs - list of dicts
    stemmed - list of lists of stemmed tokens
    returns list of ids of updated docs
    """
    max_id = redis_conn.get("max doc id")
    if max_id is None:
        max_id = 0
    else:
        max_id = int(max_id)
    new_ids = list(range(max_id, len(docs), 1))
    
    pipe = redis_conn.pipeline()
    
    stemmed = [' '.join(stem_list) for stem_list in stemmed]
    tf_idf_vectorizer = TfidfVectorizer()
    X = tf_idf_vectorizer.fit_transform(stemmed)
    
#     current_toks = set(sum(stemmed, []))
    # create existing_docs dict {token: list_of_docs_ids}
#     existing_ids = redis_conn.sinter(*tokens)
#     for tok in all_toks:
#         pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
#     existing_docs = {tok: lst for tok, lst in zip(vocabulary, pipe.execute())}

    # get old df from redis
    # old_df_dict - {tok: df} (df - len of docs in redis for token tok)
    # for tok in vocabulary:
    #     routine_pipe.zcount(tok, '-inf', '+inf')
    # buff = routine_pipe.execute()
    # old_df_dict = {tok: df for tok, df in zip(vocabulary, buff)}
    vocab = tf_idf_vectorizer.vocabulary_
    inv_vocab = {vocab[key]:key for key in vocab}
    
    cx = scipy.sparse.coo_matrix(X)
    
    for d_id, tok_id, tf_idf in zip(cx.row, cx.col, cx.data):
        pipe.zadd(inv_vocab[tok_id], d_id, tf_idf)
    pipe.execute()

    for new_doc, new_id in zip(docs, new_ids):
        pipe.hmset("doc:{}".format(new_id), new_doc)
    pipe.incrby("max doc id", len(docs))
    print(pipe.execute())
    return list(new_ids)

In [23]:
# import ipy_autoreload
%reload_ext autoreload 
%autoreload 1
%aimport paper
# from paper import index
from paper.utils import timed


@timed
def update_timed(list_json, stemmed):
    update_index_tfidf(list_json, stemmed)

redis_conn.flushall()
update_timed(list_json[:500], stemmed[:500]);

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [15]:
sum(stemmed,[])

TypeError: can only concatenate list (not "str") to list

In [12]:
pipe = redis_conn.pipeline()
all_toks = set(sum(stemmed[:500], []))
for tok in all_toks:
    pipe.zrange(tok, 0, -1, score_cast_func=float)
buff = pipe.execute()
old_table_vocab = {tok: lst for tok, lst in zip(all_toks, buff)}

In [13]:
old_table_vocab

{'': [],
 'таксист': ['239'],
 '100$': [],
 'качествен': ['373', '181', '413', '254', '207', '118', '82', '167'],
 'закупк': ['347', '269'],
 'торжеств': ['283'],
 'арх': ['362'],
 'лестниц': ['426'],
 'курсов': ['2',
  '310',
  '14',
  '222',
  '20',
  '290',
  '103',
  '218',
  '182',
  '459',
  '296'],
 'ногт': ['150'],
 'товар': ['78',
  '27',
  '285',
  '160',
  '298',
  '213',
  '396',
  '329',
  '356',
  '499',
  '248',
  '91',
  '261',
  '426',
  '168',
  '9',
  '439',
  '318',
  '112',
  '113',
  '216',
  '217',
  '181',
  '397',
  '355',
  '437',
  '269',
  '253',
  '409',
  '343',
  '349',
  '331',
  '392',
  '473',
  '58',
  '489',
  '207',
  '373',
  '61',
  '49'],
 'специальн': ['250', '11', '432'],
 'задач': ['441',
  '86',
  '240',
  '404',
  '402',
  '14',
  '393',
  '254',
  '154',
  '17',
  '398',
  '284',
  '197',
  '123',
  '496',
  '73',
  '98',
  '16',
  '200',
  '380',
  '187',
  '111',
  '452',
  '205',
  '353',
  '340'],
 'сервак': ['44'],
 '18ти': ['176'],
 '

In [ ]:
import autoreload
?autoreload

In [ ]:
X

In [ ]:
from redis import Redis


tokens = ["inv+ "+tok for tok in ['водител', 'такс']]
r = Redis()
r.sinter(*tokens)

In [ ]:
! pip install redis==2.10.6

In [ ]:
from paper.index import get_docs

get_docs([1,2])

## Make Index update via `updater` service

In [ ]:
from paper.utils import post_request as post
from paper.index import delete_all

delete_all()

update_json = {'docs': list_json[100:200]}

post(UPDATER_PORT, UPDATER_PATH, update_json)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X)
print(vectorizer.vocabulary_)